# Benchmark the reliability solvers on the problems

In this example, we show how to run all the methods on all the problems and get the computed probability.

In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd
import csv

We import the list of reliability problems.

In [2]:
benchmarkProblemList = otb.ReliabilityBenchmarkProblemList()
numberOfProblems = len(benchmarkProblemList)
numberOfProblems

26

In [3]:
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    pf = problem.getProbability()
    print("#", i, " : ", name, ", exact PF : ", pf)

# 0  :  RP8 , exact PF :  0.000784
# 1  :  RP14 , exact PF :  0.00752
# 2  :  RP22 , exact PF :  0.00416
# 3  :  RP24 , exact PF :  0.00286
# 4  :  RP25 , exact PF :  6.14e-06
# 5  :  RP28 , exact PF :  1.46e-07
# 6  :  RP31 , exact PF :  0.00018
# 7  :  RP33 , exact PF :  0.00257
# 8  :  RP35 , exact PF :  0.00354
# 9  :  RP38 , exact PF :  0.0081
# 10  :  RP53 , exact PF :  0.0313
# 11  :  RP55 , exact PF :  0.36
# 12  :  RP54 , exact PF :  0.000998
# 13  :  RP57 , exact PF :  0.0284
# 14  :  RP75 , exact PF :  0.0107
# 15  :  RP89 , exact PF :  0.00543
# 16  :  RP107 , exact PF :  2.92e-07
# 17  :  RP110 , exact PF :  3.19e-05
# 18  :  RP111 , exact PF :  7.65e-07
# 19  :  RP63 , exact PF :  0.000379
# 20  :  RP91 , exact PF :  0.000697
# 21  :  RP60 , exact PF :  0.0456
# 22  :  RP77 , exact PF :  2.87e-07
# 23  :  Four-branch serial system (Waarts, 2000) , exact PF :  0.0021859614549132322
# 24  :  R-S , exact PF :  0.07864960352514257
# 25  :  Axial stressed beam , exact PF :  0.

In [4]:
i = 3
problem = benchmarkProblemList[i]

benchmarkFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
    problem,
    nearestPointAlgo="Cobyla",
    maximumEvaluationNumber=100,
    maximumAbsoluteError=1.0e-4,
    maximumRelativeError=1.0e-3,
    maximumResidualError=1.0e-3,
    maximumConstraintError=1.0e-3,
)

benchmarkSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
    problem,
    nearestPointAlgo="Cobyla",
    maximumEvaluationNumber=100,
    maximumAbsoluteError=1.0e-4,
    maximumRelativeError=1.0e-3,
    maximumResidualError=1.0e-3,
    maximumConstraintError=1.0e-3,
)

benchmarkMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(
    problem, maximumOuterSampling=1000000, coefficientOfVariation=0.1, blockSize=1,
)

benchmarkFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(
    problem,
    nearestPointAlgo="Cobyla",
    maximumEvaluationNumber=100,
    maximumAbsoluteError=1.0e-3,
    maximumRelativeError=1.0e-3,
    maximumResidualError=1.0e-3,
    maximumConstraintError=1.0e-3,
    maximumOuterSampling=5000,
    coefficientOfVariation=0.1,
)

benchmarkSS = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
    problem, maximumOuterSampling=5000, coefficientOfVariation=0.1, blockSize=1,
)

s1 = otb.OTReliabilityAlgorithmBenchmark.printResultFORM(benchmarkFORM)
s2 = otb.OTReliabilityAlgorithmBenchmark.printResultSORM(benchmarkSORM)
s3 = otb.OTReliabilityAlgorithmBenchmark.printResultMC(benchmarkMC)
s4 = otb.OTReliabilityAlgorithmBenchmark.printResultFORMIS(benchmarkFORMIS)
s5 = otb.OTReliabilityAlgorithmBenchmark.printResultSubset(benchmarkSS)
print(s1)
print(s2)
print(s3)
print(s4)
print(s5)

computedProbability = 0.008197536156089676  absoluteError = 0.005337536156089676 numberOfCorrectDigits = -0.2709747967938827 numberOfFunctionEvaluations = 101
computedProbability = 0.008197535964320822  absoluteError = 0.005337535964320822 numberOfCorrectDigits = -0.27097478119039925 numberOfFunctionEvaluations = 101
computedProbability = 0.002858694719990857  absoluteError = 1.3052800091430984e-06 numberOfCorrectDigits = 3.340662346449938 numberOfFunctionEvaluations = 34981
computedProbability = 0.003135538951157542  absoluteError = 0.0002755389511575417 numberOfCorrectDigits = 1.0161830322023797 numberOfFunctionEvaluations = 623
computedProbability = 0.0026187520000000024  absoluteError = 0.00024124799999999773 numberOfCorrectDigits = 1.0739023114969026 numberOfFunctionEvaluations = 15000


In [5]:
problem_names = []
for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    name = problem.getName()
    problem_names.append(name)

In [6]:
metrics = [
    "Exact",
    "FORM",
    "SORM",
    "Monte Carlo",
    "FORM-IS",
    "Subset",
]
results = np.zeros((numberOfProblems, len(metrics)))

for i in range(numberOfProblems):
    problem = benchmarkProblemList[i]
    results[i][0] = problem.getProbability()
    results[i][1] = otb.OTReliabilityAlgorithmBenchmark.FORM(
        problem,
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-4,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
    )[0]
    results[i][2] = otb.OTReliabilityAlgorithmBenchmark.SORM(
        problem,
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
    )[0]
    results[i][3] = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(
        problem, maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1,
    )[0]
    results[i][4] = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(
        problem,
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=5000,
        coefficientOfVariation=0.1,
    )[0]
    results[i][5] = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        problem, maximumOuterSampling=5000, coefficientOfVariation=0.1, blockSize=1,
    )[0]

df = pd.DataFrame(results, index=problem_names, columns=metrics)
df

,Exact,FORM,SORM,Monte Carlo,FORM-IS,Subset
RP8,7.840000e-04,6.599181e-04,7.838449e-04,0.000900,7.302408e-04,8.552860e-04
RP14,7.520000e-03,4.814808e-04,5.476772e-04,0.000700,7.377099e-04,6.960000e-04
RP22,4.160000e-03,6.227215e-03,4.403739e-03,0.004400,3.845855e-03,3.870000e-03
RP24,2.860000e-03,8.197536e-03,8.197536e-03,0.002800,2.911931e-03,2.530000e-03
RP25,6.140000e-06,1.645960e-01,1.648403e-01,0.000000,0.000000e+00,4.100000e-05
RP28,1.460000e-07,8.846210e-03,1.085747e-02,0.000000,1.034011e-07,9.193576e-08
RP31,1.800000e-04,2.280418e-02,2.278172e-02,0.002500,2.989877e-03,4.025932e-03
RP33,2.570000e-03,1.354337e-03,1.354336e-03,0.002000,1.805751e-03,2.722000e-03
RP35,3.540000e-03,1.354337e-03,2.140857e-03,0.003900,2.407280e-03,3.202000e-03
RP38,8.100000e-03,2.793469e-02,2.824548e-02,0.008500,7.108421e-03,8.211544e-03


In [7]:
df.to_csv("reliability_benchmark_table-output.csv")